In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

In [2]:
import urllib3
urllib3.disable_warnings()

#tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
)

In [3]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

In [4]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [5]:
class ConvBlock(tf.keras.Model):
    def __init__(self, channel, kernel_size, strides = 1):
        super(ConvBlock,self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(channel, kernel_size=kernel_size, strides=strides, padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        
    def call(self, input_tensor, training = False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training = training)
        return x

In [6]:
class SELayer(tf.keras.layers.Layer):
    def __init__(self,out_channel =64, r_ratio=4):
        super(SELayer,self).__init__()
        self.gap = tf.keras.layers.GlobalAveragePooling2D()
        self.fc1 = tf.keras.layers.Dense(out_channel//r_ratio)
        self.fc2 = tf.keras.layers.Dense(out_channel)
        self.output_channel = out_channel
        self.reshape = tf.keras.layers.Reshape((1,1,out_channel))
    
    def call(self,input_tensor,training = False):
        x = self.gap(input_tensor)
        x = self.fc1(x)
        x = tf.nn.relu(x)
        x = self.fc2(x)
        x = tf.nn.sigmoid(x)
        x = self.reshape(x)
        return x*input_tensor

In [7]:
class SEBlock(tf.keras.layers.Layer):
    def __init__(self,channel,down_sample = False, r_ratio=4):
        super(SEBlock,self).__init__()
        self.down_sample = down_sample
        
        if self.down_sample:
            self.conv_block1 = ConvBlock(channel,kernel_size=3, strides=2)
        else:
            self.conv_block1 = ConvBlock(channel,kernel_size=3,  strides=1)
        
        self.conv_block2 = ConvBlock(channel,kernel_size=3,  strides=1)
        
        self.se1 = SELayer(channel,r_ratio)
                
        if self.down_sample:
            self.downsample_conv = tf.keras.layers.Conv2D(channel, kernel_size=1, strides=2, padding='same')
            self.downsample_bn = tf.keras.layers.BatchNormalization()
                
    def call(self,input_tensor,training = False):
        x = self.conv_block1(input_tensor)
        x = tf.nn.relu(x)
        
        x = self.conv_block2(x)
        x = self.se1(x)
        
        if self.down_sample:
            input_tensor = self.downsample_conv(input_tensor)
            input_tensor = self.downsample_bn(input_tensor, training= training)
        
        x = x + input_tensor
        x = tf.nn.relu(x)
        return x

In [8]:
class SE_ResNet34(tf.keras.Model):
    def __init__(self):
        super(SE_ResNet34,self).__init__()
        
        self.conv = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')
        self.bn = tf.keras.layers.BatchNormalization()     
        self.maxpool = tf.keras.layers.MaxPool2D(pool_size=2)
               
        self.conv1_1 = SEBlock(64, down_sample = True)
        self.conv1_2 = SEBlock(64)
        self.conv1_3 = SEBlock(64)

        self.conv2_1 = SEBlock(128, down_sample = True)
        self.conv2_2 = SEBlock(128)
        self.conv2_3 = SEBlock(128)      
        self.conv2_4 = SEBlock(128)      
        
        self.conv3_1 = SEBlock(256, down_sample = True)
        self.conv3_2 = SEBlock(256)
        self.conv3_3 = SEBlock(256)      
        self.conv3_4 = SEBlock(256)           
        self.conv3_5 = SEBlock(256)      
        self.conv3_6 = SEBlock(256)      
        
        self.conv4_1 = SEBlock(512, down_sample = True)
        self.conv4_2 = SEBlock(512)
        self.conv4_3 = SEBlock(512)              
        
        self.gap = tf.keras.layers.GlobalAveragePooling2D()
        self.fc1 = tf.keras.layers.Dense(4096, activation='relu')
        self.fc2 = tf.keras.layers.Dense(4096, activation='relu')
        self.fc3 = tf.keras.layers.Dense(10, activation='softmax')        
        
    def call(self, input_tensor, training = False):
        x = self.conv(input_tensor)
        x = self.bn(x)
        x = tf.nn.relu(x)
        x = self.maxpool(x)
        
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.conv1_3(x)
        
        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.conv2_3(x)
        x = self.conv2_4(x)
        
        x = self.conv3_1(x)
        x = self.conv3_2(x)
        x = self.conv3_3(x)
        x = self.conv3_4(x)
        x = self.conv3_5(x)
        x = self.conv3_6(x)
        
        x = self.conv4_1(x)
        x = self.conv4_2(x)
        x = self.conv4_3(x)
        
        x = self.gap(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [9]:
BATCH_SIZE = 256
EPOCH = 20

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

In [10]:
se_resnet34 = SE_ResNet34()

se_resnet34.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_se_resnet34 = se_resnet34.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1
)

Epoch 1/20
195/195 [==============================] - 14s 70ms/step - loss: 1.9758 - accuracy: 0.2765 - val_loss: 2.3375 - val_accuracy: 0.1005
Epoch 2/20
195/195 [==============================] - 13s 66ms/step - loss: 1.5864 - accuracy: 0.4265 - val_loss: 2.3020 - val_accuracy: 0.1724
Epoch 3/20
195/195 [==============================] - 12s 61ms/step - loss: 1.4138 - accuracy: 0.4879 - val_loss: 1.6791 - val_accuracy: 0.4000
Epoch 4/20
195/195 [==============================] - 12s 63ms/step - loss: 1.2702 - accuracy: 0.5462 - val_loss: 1.5029 - val_accuracy: 0.4660
Epoch 5/20
195/195 [==============================] - 12s 62ms/step - loss: 1.1154 - accuracy: 0.6069 - val_loss: 1.5001 - val_accuracy: 0.4709
Epoch 6/20
195/195 [==============================] - 12s 61ms/step - loss: 0.9845 - accuracy: 0.6583 - val_loss: 1.5641 - val_accuracy: 0.4699
Epoch 7/20
195/195 [==============================] - 12s 63ms/step - loss: 0.8100 - accuracy: 0.7241 - val_loss: 1.6648 - val_accuracy: